 <img src="so_logo.png">
 
# Analysis of Stack Overflow

In [1]:
## Author -> Sanath Shivaswamy
## ID -> W1166731

In [3]:
# List of Packages
import pandas as pd
import numpy as np
import random
import sklearn
import rake
import operator
import time
import math
import itertools
import nltk
import re
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from bs4 import BeautifulSoup
from IPython.display import display, HTML
from plotly.tools import FigureFactory as FF
from collections import defaultdict, OrderedDict
#plotly.tools.set_credentials_file(username='sshivaswamy', api_key='lGR8CEhhJmHISOzPz8uV')
plotly.tools.set_credentials_file(username='surbhi.gupta', api_key='Yg3oYlMgdEUNrJMqEuXy')
print "All Packages Installed"

All Packages Installed


 <img src="so-icon.png" style="float:left">
 ## Problem Statement
    1. Predicting Tags for StackOverflow Posts
    2. Analyze Factors that influence Reponse Time

  <img src="so-icon.png" style="float:left">
  ## Application to our Analysis
    1. Tag Recommendation to Users and Groups based on their interests
    2. Tag Cleanup Systems - Mistags are identified and corrected
    


<img src="data_explorer.png" style="float:left">

## StackOverflow Dictionary
1. PostType
2. AcceptedAnswer
3. Title
4. Tags
5. Body
6. AnswerCount
7. View Count
8. Comments Count
9. Reputation

In [10]:
posts = pd.read_csv("/Users/sanath.shivaswamy/Desktop/StackOverflow/Posts.csv")
users = pd.read_csv("/Users/sanath.shivaswamy/Desktop/StackOverflow/Users.csv")
badges = pd.read_csv("/Users/sanath.shivaswamy/Desktop/StackOverflow/Badges.csv")
comments = pd.read_csv("/Users/sanath.shivaswamy/Desktop/StackOverflow/Comments.csv")
poststypes = pd.read_csv("/Users/sanath.shivaswamy/Desktop/StackOverflow/PostsTypes.csv")
tags = pd.read_csv("/Users/sanath.shivaswamy/Desktop/StackOverflow/Tags.csv")

In [5]:
print "Posts = ", posts.shape
print "Users = ", users.shape
print "Badges = ", badges.shape
print "Comments = ", comments.shape
print "Posttypes = ", poststypes.shape
print "Tags = ", tags.shape

Posts =  (50000, 22)
Users =  (50000, 15)
Badges =  (50000, 6)
Comments =  (50000, 7)
Posttypes =  (8, 2)
Tags =  (47207, 5)


In [8]:
print posts.columns
posts.head(3)

Index([u'Id', u'PostTypeId', u'AcceptedAnswerId', u'ParentId', u'CreationDate',
       u'DeletionDate', u'Score', u'ViewCount', u'Body', u'OwnerUserId',
       u'OwnerDisplayName', u'LastEditorUserId', u'LastEditorDisplayName',
       u'LastEditDate', u'LastActivityDate', u'Title', u'Tags', u'AnswerCount',
       u'CommentCount', u'FavoriteCount', u'ClosedDate',
       u'CommunityOwnedDate'],
      dtype='object')


,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,32719536,1,NaN,NaN,2015-09-22 14:25:31,NaN,6,165.0,<p>This code is correct C++:</p>\n\n<pre><code...,193789.0,...,NaN,2015-09-22 20:10:46,2015-09-22 20:10:46,Why is namespace std not required for legacy C...,<c++>,1.0,6,1.0,NaN,NaN
1,32719537,2,NaN,32718671.0,2015-09-22 14:25:32,NaN,2,NaN,"<p>can you try </p>\n\n<pre><code> $(""#_LOAD""...",4537150.0,...,NaN,NaN,2015-09-22 14:25:32,NaN,NaN,NaN,3,NaN,NaN,NaN
2,32719538,2,NaN,32719399.0,2015-09-22 14:25:41,NaN,0,NaN,<p>You should cast your str var to a bytes var...,3501059.0,...,NaN,2015-09-23 10:59:50,2015-09-23 10:59:50,NaN,NaN,NaN,3,NaN,NaN,NaN


In [9]:
print users.columns
users.head(3)

Index([u'Id', u'Reputation', u'CreationDate', u'DisplayName',
       u'LastAccessDate', u'WebsiteUrl', u'Location', u'AboutMe', u'Views',
       u'UpVotes', u'DownVotes', u'ProfileImageUrl', u'EmailHash', u'Age',
       u'AccountId'],
      dtype='object')


,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,EmailHash,Age,AccountId
0,3904458,1,2014-08-03 18:13:34,Iliya,2014-09-18 05:45:30,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/?s=128&d=ident...,NaN,NaN,4838749
1,3904459,1,2014-08-03 18:13:49,Nguyễn Huy Quyết,2016-11-02 03:39:51,NaN,NaN,NaN,0,0,0,https://graph.facebook.com/100001678204894/pic...,NaN,NaN,4838750
2,3904460,1,2014-08-03 18:13:58,user3904460,2014-08-03 18:13:58,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/?s=128&d=ident...,NaN,NaN,4838751


Picking only those posts that have an accepted answer

In [11]:
#### PostypeId = 1 --> Question, 2 -> Answer
questions = posts.merge(posts, left_on='AcceptedAnswerId', right_on='Id', how='inner') 
print questions.shape

(3207, 44)


In [12]:
questions_text = questions.ix[:,[0,8,15,16]]
questions_text.columns = ["Id","Body","Title","Tags"]
questions_text.head(3)

,Id,Body,Title,Tags
0,32719590,<p>How do I bring back a second name field fro...,SQL - Return field from sub sub query,<sql><sql-server><join><subquery>
1,32719623,<p>I have a JavaScript function where I change...,jQuery does not detect current class,<javascript><jquery><css>
2,17537590,<p>When trying to output an element of a vecto...,Symbol Not Found when Outputting Vector,<c++>


# Cleaning Text
1. Remove everything inside brackets
2. Take only nouns and remove verbs and adjectives
3. Remove special characters

In [21]:
print questions_text.loc[1,"Body"]

<p>I have a JavaScript function where I change the class of a button (<code>&lt;a href="..." class="button" id="placeOrdersButton"&gt;Button&lt;/a&gt;</code>):</p>

<pre><code>function showOrderButtonActive(isActive) {
        if (isActive) {
            $("#placeOrdersButton").addClass("button");
            $("#placeOrdersButton").removeClass("buttonDisabled");
        } else {
            $("#placeOrdersButton").removeClass("button");
            $("#placeOrdersButton").addClass("buttonDisabled");
        }
</code></pre>

<p>I also have a JavaScript function to show a loading panel when clicking on enabled buttons:</p>

<pre><code>$("a:not([target='_blank'], [class='buttonDisabled'])").on("click", function (e) {
        window.LoadingPanel.Show();
    });
</code></pre>

<p>But when my JavaScript function changes the class to <code>buttonDisabled</code> it still shows the loading panel.</p>

<p>Why does jQuery not detect the current class name and how can I do this the best and most 

In [22]:
stop_words = {'a', "a's", 'able', 'about', 'above', 'according', 'accordingly','across', 'actually', 'after', 'afterwards', 'again', 'against',
              "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along','already', 'also', 'although', 'always', 'am', 'among', 'amongst',
              'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone','anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear',
              'appreciate', 'appropriate', 'are', "aren't", 'around', 'as','aside', 'ask', 'asking', 'associated', 'at', 'available', 'away',
              'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes','becoming', 'been', 'before', 'beforehand', 'behind', 'being',
              'believe', 'below', 'beside', 'besides', 'best', 'better','between', 'beyond', 'both', 'brief', 'but', 'by', "c'mon",
              "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause','causes', 'certain', 'certainly', 'changes', 'clearly', 'co',
              'com', 'come', 'comes', 'concerning', 'consequently', 'consider','considering', 'contain', 'containing', 'contains',
              'corresponding', 'could', "couldn't", 'course', 'currently', 'd','definitely', 'described', 'despite', 'did', "didn't",
              'different', 'do', 'does', "doesn't", 'doing', "don't", 'done','down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight',
              'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially','et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone',
              'everything', 'everywhere', 'ex', 'exactly', 'example', 'except','f', 'far', 'few', 'fifth', 'first', 'five', 'followed',
              'following', 'follows', 'for', 'former', 'formerly', 'forth','four', 'from', 'further', 'furthermore', 'g', 'get', 'gets',
              'getting', 'given', 'gives', 'go', 'goes', 'going', 'gone', 'got','gotten', 'greetings', 'h', 'had', "hadn't", 'happens', 'hardly',
              'has', "hasn't", 'have', "haven't", 'having', 'he', "he's",'hello', 'help', 'hence', 'her', 'here', "here's", 'hereafter',
              'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him','himself', 'his', 'hither', 'hopefully', 'how', 'howbeit',
              'however', 'i', "i'd", "i'll", "i'm", "i've", 'ie', 'if','ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed',
              'indicate', 'indicated', 'indicates', 'inner', 'insofar','instead', 'into', 'inward', 'is', "isn't", 'it', "it'd", "it'll",
              "it's", 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps','kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later',
              'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's",'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks',
              'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean','meanwhile', 'merely', 'might', 'more', 'moreover', 'most',
              'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely','nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither',
              'never', 'nevertheless', 'new', 'next', 'nine', 'no', 'nobody','non', 'none', 'noone', 'nor', 'normally', 'not', 'nothing',
              'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often','oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only',
              'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our','ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own',
              'p', 'particular', 'particularly', 'per', 'perhaps', 'placed','please', 'plus', 'possible', 'presumably', 'probably',
              'provides', 'q', 'que', 'quite', 'qv', 'rather', 'rd', 're','really', 'reasonably', 'regarding', 'regardless', 'regards',
              'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw','say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing',
              'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves','sensible', 'sent', 'serious', 'seriously', 'seven', 'several',
              'shall', 'she', 'should', "shouldn't", 'since', 'six', 'so','some', 'somebody', 'somehow', 'someone', 'something', 'sometime',
              'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry','specified', 'specify', 'specifying', 'still', 'sub', 'such',
              'sup', 'sure', 't', "t's", 'take', 'taken', 'tell', 'tends', 'th','than', 'thank', 'thanks', 'thanx', 'that', "that's", 'thats',
              'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence','there', "there's", 'thereafter', 'thereby', 'therefore',
              'therein', 'theres', 'thereupon', 'these', 'they', "they'd","they'll", "they're", "they've", 'think', 'third', 'this',
              'thorough', 'thoroughly', 'those', 'though', 'three', 'through','throughout', 'thru', 'thus', 'to', 'together', 'too', 'took',
              'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying','twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless',
              'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used','useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value',
              'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants','was', "wasn't", 'way', 'we', "we'd", "we'll", "we're", "we've",
              'welcome', 'well', 'went', 'were', "weren't", 'what', "what's",'whatever', 'when', 'whence', 'whenever', 'where', "where's",
              'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon','wherever', 'whether', 'which', 'while', 'whither', 'who',
              "who's", 'whoever', 'whole', 'whom', 'whose', 'why', 'will','willing', 'wish', 'with', 'within', 'without', "won't", 'wonder',
              'would', 'would', "wouldn't", 'x', 'y', 'yes', 'yet', 'you',"you'd", "you'll", "you're", "you've", 'your', 'yours',
              'yourself', 'yourselves', 'z', 'zero'}

In [23]:
# Remove HTML tags and clean the body
def cleanhtml(raw_html):
    text = ""
    text = raw_html.lower()
    text = BeautifulSoup(text, "html").text # to remove html tags
    #cleanr = re.compile('<.*?>')
    #text = re.sub(cleanr, '', text)
    #cleanr = re.compile('{.*}')
    #text = re.sub(r'\([^)]*\)', '', text)
    #text = re.sub(cleanr, '', text)
    cleanr = re.compile("[^\w']|_") #Removing special characters
    text = re.sub(cleanr, ' ', text)
    result = ''.join([i for i in text if not i.isdigit()]) #Removing numbers
    return remove_stop_words(result)

In [24]:
cachedStopWords = stop_words
def remove_stop_words(sentence):
    word_list = sentence.split()
    new_list = []
    for word in word_list:
        word = word.lower()
        if word == "c" or word == "r":
            new_list.append(word)
        if len(word) <= 10 and len(word) > 2:
            new_list.append(word)
    new_list = [word for word in new_list if word not in cachedStopWords]
    return remove_extra_words(new_list)

In [25]:
extra_words = ["function","class","endl","dont","don't","id","work","rethink","solutions","college","show","see","others","skills","colors","name","id","good","hope","knowledge","multiple","curiosity","good","help","pretty","question","sense","lessons","forum","moon","use","need","guys","stuff","way","changes","int","class","float","make","variables","with","will","didn't","values","know","page","comments","clear","think","year","indicating","using","making","allow","allowing","double","compare","value","var","vars","id","didn'","put","change","input","switch","case","if","else"]
def remove_extra_words(word_list):
    clean_text=""
    word_list = list(set(word_list))
    for word in word_list:
        clean_text = ' '.join([word for word in word_list if word not in extra_words]) 
    #return get_nouns(nouns_text)
    return clean_text

In [26]:
# Take only nouns from the text
# Using this function reduced the accuracy
def get_nouns(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    nouns=[]
    downcased = []
    joined = ""
    for word,pos in tagged:
        if pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS':
            nouns.append(word)
    downcased = [x.lower() for x in nouns]
    joined = " ".join(downcased).encode('utf-8')    
    return joined

In [27]:
for index,row in questions_text.iterrows():
    temp = questions_text.loc[index,"Body"] 
    temp = cleanhtml(temp)
    questions_text.loc[index,"Body"] = temp
print "Text cleaning Complete"

/Users/sanath.shivaswamy/Desktop/datascience2016fall/venv/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")


/Users/sanath.shivaswamy/Desktop/datascience2016fall/venv/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

Text cleaning Complete


# Predict StackOverflow Tags
 Different Approaches:

    1. RAKE (Rapid Automatic Keyword Extraction)
    
    2. TF-IDF (Inverse Document Frequency)
    
    3. Simple Word Frequency Model
    

## 1st Approach: RAKE  (Rapid Automatic Keyword Extraction)

In [99]:
rake_object = rake.Rake("/Users/sanath.shivaswamy/Desktop/StackOverflow/SmartStoplist.txt", 1, 1, 1)
result_table = pd.DataFrame()
for i in range(5):
    text = questions_text.loc[i].Title
    text = text + questions_text.loc[i].Body
    predicted_column = rake_object.run(text)
    prediction_list = []
    prediction_list.append(predicted_column)
    result_table.loc[i,"Predicted"] = prediction_list
    result_table.loc[i,"Actual"] = questions_text.loc[i].Tags
table = FF.create_table(result_table)
py.iplot(table)

In [29]:
def tag_accuracy(column):
    True_Positive = 0
    False_Negative = 0
    Total = 0
    # Creating a Rake object
    rake_object = rake.Rake("/Users/sanath.shivaswamy/Desktop/StackOverflow/SmartStoplist.txt", 3, 3, 1)

    for index,row in questions_text.iterrows():
        Total += 1
        temp=[]
        if column == "Both":
            text = questions_text.loc[index].Title
            text = text.decode('utf-8')
            text = text + questions_text.loc[index].Body
            candidates = rake_object.run(text)
        elif column == "Title":
            candidates = rake_object.run(questions_text.loc[index].Title)
        elif column == "Body":
            candidates = rake_object.run(questions_text.loc[index].Body)
        tag_candidates = [key[0] for key in candidates]
        if len(tag_candidates) == 0:
            tag_candidates.append("none")
        tag_present = False
        for tag in tag_candidates:
            if tag in questions_text.loc[index].Tags:
                tag_present = True
        if tag_present:
            True_Positive += 1
        else:
            False_Negative += 1

    return True_Positive,False_Negative

In [30]:
Title_TP,Title_FN = tag_accuracy("Title")
Accuracy = float(Title_TP) / float(Title_TP + Title_FN)
print "TP = ", Title_TP
print "FN = ", Title_FN
print "Accuracy =" , Accuracy

TP =  1264
FN =  1943
Accuracy = 0.394137823511


In [31]:
Body_TP,Body_FN = tag_accuracy("Body")
Accuracy = float(Body_TP) / float(Body_TP + Body_FN)
print "TP = ", Body_TP
print "FN = ", Body_FN
print "Accuracy =" , Accuracy

TP =  50
FN =  3157
Accuracy = 0.0155908949174


In [32]:
Both_TP,Both_FN = tag_accuracy("Both")
Accuracy = float(Both_TP) / float(Both_TP + Both_FN)
print "TP = ", Both_TP
print "FN = ", Both_FN
print "Accuracy =" , Accuracy

TP =  978
FN =  2229
Accuracy = 0.304957904584


In [71]:
# True Positive and False Negative charts for Title and Description based Tag Prediction
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    y=['Title & Body', 'Body', 'Title'],
    x=[Both_TP, Body_TP, Title_TP],
    name='True Positive',
    orientation = 'h',
    marker = dict(
        color = 'rgba(255, 0, 0, 0.4)',
        line = dict(
            color = 'rgba(246, 78, 139, 1.0)',
            width = 3)
    )
)
trace2 = go.Bar(
    y=['Title & Body', 'Body', 'Title'],
    x=[Both_FN, Body_FN, Title_FN],
    name='False Negative',
    orientation = 'h',
    marker = dict(
        color = 'rgba(58, 71, 80, 0.6)',
        line = dict(
            color = 'rgba(58, 71, 80, 1.0)',
            width = 3)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)

In [72]:
py.iplot(fig)

## 2nd Approach: TF-IDF (Inverse Document Frequency)

#### TF(t) : 
(No of times term t appears in a doc) / (Total no of terms in the doc)

#### IDF(t) : 
log_e(Total no of documents / No of documents with term t)

In [34]:
def process_text(doc, idf, text):
    tf = OrderedDict()
    word_count = 0.
    for word in text.split():
        word = word.lower()
        if word not in stop_words and word.isalpha():
            if word not in tf:
                tf[word] = 0
            tf[word] += 1
            idf[word].add(doc)
            word_count += 1.
    for word in tf:
        tf[word] = tf[word] / word_count
    return tf, word_count

docs = []
# Calculate TF and IDF per document
idf = defaultdict(set)
tf = {}
word_counts = defaultdict(float)
for index,row in questions_text.iterrows():
    doc = int(row["Id"])
    docs.append(doc)
    text = ""
    text = questions_text.loc[index].Title + ' ' 
    text = text.decode('utf-8')
    text = text + questions_text.loc[index].Body
    tf[doc], word_counts[doc] = process_text(doc, idf, text)
# Calculate TF-IDF
nr_docs = len(docs)
for doc in docs:
    for word in tf[doc]:
        tf[doc][word] *= math.log(nr_docs / len(idf[word]))
        
# Making predictions
True_positives = 0
False_negatives = 0
result_table = pd.DataFrame()
counter = 0
for doc in docs:
    # Sort words with frequency from high to low.
    pred_tags = sorted(tf[doc], key=tf[doc].get, reverse=True)[:3]
    result_table.loc[counter,"Predicted"] = " ,".join(sorted(pred_tags))
    result_table.loc[counter,"Actual"] = list(questions_text[(questions_text["Id"]==doc)]["Tags"])
    counter += 1
    present = False
    for tag in pred_tags:
        if tag in str(questions_text[(questions_text["Id"]==doc)]["Tags"]):
            present = True
    if present:
        True_positives += 1
    else:
        False_negatives += 1

In [35]:
print "TP = ",True_positives
print "FN = ",False_negatives
Accuracy = float(True_positives) / float(True_positives + False_negatives)
print "Accuracy = ",Accuracy

TP =  1327
FN =  1880
Accuracy =  0.413782351107


In [36]:
table = FF.create_table(result_table[50:55])
py.iplot(table)

## 3rd Approach: Word Frequency Model

In [37]:
def process_text(doc, text):
    tf = OrderedDict()
    word_count = 0.
    for word in text.split():
        word = word.lower()
        if word not in stop_words and word.isalpha():
            if word not in tf:
                tf[word] = 0
            tf[word] += 1
            word_count += 1.
    for word in tf:
        tf[word] = tf[word] / word_count
    return tf, word_count

docs = []
# Calculate TF per document
idf = defaultdict(set)
tf = {}
word_counts = defaultdict(float)
for index,row in questions_text.iterrows():
    doc = int(row["Id"])
    docs.append(doc)
    text = ""
    text = questions_text.loc[index].Title + ' ' 
    text = text.decode('utf-8')
    text = text + questions_text.loc[index].Body
    tf[doc], word_counts[doc] = process_text(doc, text)
    
# Making predictions
True_positives = 0
False_negatives = 0
result_table = pd.DataFrame()
counter = 0
for doc in docs:
    # Sort words with frequency from high to low.
    pred_tags = sorted(tf[doc], key=tf[doc].get, reverse=True)[:3]
    result_table.loc[counter,"Predicted"] = " ,".join(sorted(pred_tags))
    result_table.loc[counter,"Actual"] = list(questions_text[(questions_text["Id"]==doc)]["Tags"])
    counter += 1
    present = False
    for tag in pred_tags:
        if tag in str(questions_text[(questions_text["Id"]==doc)]["Tags"]):
            present = True
    if present:
        True_positives += 1
    else:
        False_negatives += 1

In [38]:
print "TP = ",True_positives
print "FN = ",False_negatives
Accuracy = float(True_positives) / float(True_positives + False_negatives)
print "Accuracy = ",Accuracy

TP =  1984
FN =  1223
Accuracy =  0.618646710321


In [110]:
table = FF.create_table(result_table[:5])
py.iplot(table)

## Factors that Influence Response Time of the Post 

In [39]:
temp_questions = questions.copy()
for index,row in temp_questions.iterrows():
    q_time = temp_questions.loc[index,"CreationDate_x"] 
    temp_questions.loc[index,"CreationDate_x"] = time.mktime(time.strptime(q_time, "%Y-%m-%d %H:%M:%S"));
    a_time = temp_questions.loc[index,"CreationDate_y"] 
    temp_questions.loc[index,"CreationDate_y"] = time.mktime(time.strptime(a_time, "%Y-%m-%d %H:%M:%S"));
    diff = temp_questions.loc[index,"CreationDate_y"] - temp_questions.loc[index,"CreationDate_x"]
    mins = float(diff)/60
    hours = float(mins)/60
    temp_questions.loc[index,"Response_Time"] = hours

In [41]:
analyze_data = temp_questions.ix[:,[0,6,7,17,18,44]]
analyze_data.columns = ["Id","q_score","view_count","answer_count","comment_count","response_time"]
analyze_data.head()

,Id,q_score,view_count,answer_count,comment_count,response_time
0,32719590,0,45.0,1.0,1,0.042500
1,32719623,0,80.0,1.0,1,0.078056
2,17537590,2,59.0,1.0,4,0.082500
3,17537613,9,3233.0,1.0,0,0.045278
4,17537620,0,455.0,1.0,0,0.087500


In [42]:
print "Average Response Time = ",analyze_data["response_time"].mean(),"hours"

Average Response Time =  0.755346897412 hours


In [73]:
d = analyze_data["response_time"]
d = d.drop(d.index[892]) # removing anomaly value for better visualization
trace = go.Scatter(
    name = "Hours",
    y = d,
    mode = 'markers'
)
layout= go.Layout(
    title= 'Response Time',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Index',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Hours',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend=True
)
fig = go.Figure(data=[trace], layout=layout)

In [74]:
py.iplot(fig)

The response times are usually small 

In [45]:
analyze_data.corr()

,Id,q_score,view_count,answer_count,comment_count,response_time
Id,1.000000,-0.139076,-0.140850,-0.345609,0.164619,0.002245
q_score,-0.139076,1.000000,0.896800,0.465201,0.028034,-0.008628
view_count,-0.140850,0.896800,1.000000,0.464315,0.031041,-0.010365
answer_count,-0.345609,0.465201,0.464315,1.000000,0.012539,-0.013839
comment_count,0.164619,0.028034,0.031041,0.012539,1.000000,0.002252
response_time,0.002245,-0.008628,-0.010365,-0.013839,0.002252,1.000000


## Relation between User Reputation and Response Time

In [46]:
print users.dtypes

Id                   int64
Reputation           int64
CreationDate        object
DisplayName         object
LastAccessDate      object
WebsiteUrl          object
Location            object
AboutMe             object
Views                int64
UpVotes              int64
DownVotes            int64
ProfileImageUrl     object
EmailHash          float64
Age                float64
AccountId            int64
dtype: object


In [50]:
reputation = temp_questions.merge(users, left_on='OwnerUserId_x', right_on='AccountId', how='inner') 
reputation.shape

(142, 60)

In [53]:
reputation.columns

Index([u'Id_x', u'PostTypeId_x', u'AcceptedAnswerId_x', u'ParentId_x',
       u'CreationDate_x', u'DeletionDate_x', u'Score_x', u'ViewCount_x',
       u'Body_x', u'OwnerUserId_x', u'OwnerDisplayName_x',
       u'LastEditorUserId_x', u'LastEditorDisplayName_x', u'LastEditDate_x',
       u'LastActivityDate_x', u'Title_x', u'Tags_x', u'AnswerCount_x',
       u'CommentCount_x', u'FavoriteCount_x', u'ClosedDate_x',
       u'CommunityOwnedDate_x', u'Id_y', u'PostTypeId_y',
       u'AcceptedAnswerId_y', u'ParentId_y', u'CreationDate_y',
       u'DeletionDate_y', u'Score_y', u'ViewCount_y', u'Body_y',
       u'OwnerUserId_y', u'OwnerDisplayName_y', u'LastEditorUserId_y',
       u'LastEditorDisplayName_y', u'LastEditDate_y', u'LastActivityDate_y',
       u'Title_y', u'Tags_y', u'AnswerCount_y', u'CommentCount_y',
       u'FavoriteCount_y', u'ClosedDate_y', u'CommunityOwnedDate_y',
       u'Response_Time', u'Id', u'Reputation', u'CreationDate', u'DisplayName',
       u'LastAccessDate', u'Website

In [96]:
d = reputation.copy()
d = d.drop(d.index[90])# removing anomaly value for better visualization
d = d.drop(d.index[14])
trace = go.Scatter(
    name = "Hours",
    x = d["Reputation"],
    y = d["Response_Time"],
    mode = 'markers'
)
layout= go.Layout(
    title= 'Response Time VS Reputation',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Reputation',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Hours',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend=False
)
fig = go.Figure(data=[trace], layout=layout)

In [97]:
py.iplot(fig)

In [55]:
reputation.ix[:,[44,46]].corr()

,Response_Time,Reputation
Response_Time,1.000000,0.007904
Reputation,0.007904,1.000000


### Relation between Tag Popularity and Response Time 

In [79]:
tags.head()

,Id,TagName,Count,ExcerptPostId,WikiPostId
0,74890,canonical-quickly,35,8160301.0,8160300.0
1,74892,path-iterator,8,NaN,NaN
2,74895,multidrop-bus,7,18912141.0,18912140.0
3,74896,detailview,192,36773157.0,36773156.0
4,74901,apache-tomee,109,8165096.0,8165095.0


In [80]:
tags.shape

(47207, 5)

In [81]:
popularity = temp_questions.ix[:,[0,16,44]]
popularity.columns = ["Id","Tags","Response_Time"]
popularity.head()

,Id,Tags,Response_Time
0,32719590,<sql><sql-server><join><subquery>,0.042500
1,32719623,<javascript><jquery><css>,0.078056
2,17537590,<c++>,0.082500
3,17537613,<python><python-2.7><numpy><numerical><fractions>,0.045278
4,17537620,<asp.net>,0.087500


In [90]:
tags = tags.dropna()

In [ ]:
temp_tag = tags["TagName"]
for index,row in popularity.iterrows():
    t = questions_text.loc[index,"Tags"]
    tag_list = t.split("<")
    tag_list = "".join([word for word in tag_list]).split(">")
    tag_list = filter(None, tag_list)
    tag_popularity = []
    for tag in temp_tag:
        if str(tag) in tag_list:
            temp_row = tags[(tags["TagName"]==tag)]
            if len(temp_row["Count"]) < 1:
                temp_row["Count"] = 0
            tag_popularity.append(int(temp_row["Count"]))
    a = pd.DataFrame(tag_popularity)
    popularity.loc[index,"Popularity_Score"] = int(a.max())

In [92]:
popularity.head()

,Id,Tags,Response_Time,Popularity_Score
0,32719590,<sql><sql-server><join><subquery>,0.042500,360145.0
1,32719623,<javascript><jquery><css>,0.078056,1264314.0
2,17537590,<c++>,0.082500,483603.0
3,17537613,<python><python-2.7><numpy><numerical><fractions>,0.045278,662794.0
4,17537620,<asp.net>,0.087500,303911.0


In [93]:
trace = go.Scatter(
    name = "Hours",
    y = popularity["Response_Time"],
    x = popularity["Popularity_Score"],
    mode = 'markers'
)
layout= go.Layout(
    title= 'Response Time VS Tag Popularity',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Popularity',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Hours',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend=False
)
fig = go.Figure(data=[trace], layout=layout)

In [94]:
py.iplot(fig)

In [95]:
popularity.corr()

,Id,Response_Time,Popularity_Score
Id,1.000000,0.002245,0.168440
Response_Time,0.002245,1.000000,-0.020338
Popularity_Score,0.168440,-0.020338,1.000000


# Packages Used
1. pandas
2. numpy
3. scikit-learn
4. rake
5. nltk
6. plotly
7. rise 



# Take Away
1. Simple Word Frequency Model was better at predicting tags
2. Title was a better indicator of tags than Body
3. View Count, Answer Count and Tag Popularity have negative correlation with Response Time 
4. User Reputation has positive correlation with Response Time


# Thank You